This is a test2

In [0]:
%sql
SELECT * FROM samples.tpcds_sf1.call_center

In [0]:
%run /Workspace/Shared/dbx_free/Utility/utils

In [0]:
# Define source & target catalogs
source_catalog = "samples"
target_catalog = "bronze"
schema = "tpcds_sf1"

# Optional: define merge keys per table (if not all tables use the same)
merge_key_map = {
    "call_center": ["cc_call_center_sk"],
    "catalog_returns": ["cr_item_sk", "cr_order_number"],
    "catalog_sales": ["cs_item_sk", "cs_order_number"],
    "customer": ["c_customer_sk"],
    "customer_address": ["ca_address_sk"],
    "customer_demographics": ["cd_demo_sk"],
    "date_dim": ["d_date_sk"],
    "household_demographics": ["hd_demo_sk"],
    "income_band": ["ib_income_band_sk"],
    "inventory": ["inv_item_sk", "inv_warehouse_sk"],
    "item": ["i_item_sk"],
    "promotion": ["p_promo_sk"],
    "reason": ["r_reason_sk"],
    "ship_mode": ["sm_ship_mode_sk"],
    "store": ["s_store_sk"],
    "store_returns": ["sr_item_sk", "sr_ticket_number"],
    "store_sales": ["ss_item_sk", "ss_ticket_number"],
    "time_dim": ["t_time_sk"],
    "warehouse": ["w_warehouse_sk"],
    "web_page": ["wp_web_page_sk"],
    "web_returns": ["wr_item_sk", "wr_order_number"],
    "web_sales": ["ws_item_sk", "ws_order_number"],
    "web_site": ["web_site_sk"],
    "catalog_page": ["cp_catalog_page_sk"]
}


In [0]:
tables = [row.tableName for row in spark.sql(f"SHOW TABLES IN {source_catalog}.{schema}").collect()]
print(f"Schema {schema} → {len(tables)} tables")



In [0]:
# Make sure schema exists in target catalog
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog}.{schema}")

for table in tables:
    source_table = f"{source_catalog}.{schema}.{table}"
    target_table = f"{target_catalog}.{schema}.{table}"
    
    # Pick merge keys (table-specific or fallback)
    keys = merge_key_map.get(table, default_key)
    
    # Call util function
    upsert_table(source_table, target_table, keys)

